<center><h1>第六章 连接</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 一、关系型连接
### 1. 连接的基本概念

把两张相关的表按照某一个或某一组键连接起来是一种常见操作，

在关系型连接中:

$\color{red}{键}$是十分重要的，往往用`on`参数表示。

另一个重要的要素是**连接的形式**

在`pandas`中的关系型连接函数`merge`和`join`:

提供了`how`参数来代表连接形式
> 分为左连接`left`
>
> 右连接`right`: 右表出现的扩展
>
> 内连接`inner`：同时出现的
>
> 外连接`outer`：全连接

![图片1](https://www.jianshu.com/p/23a2f0804b82)


若出现重复的键应该如何处理：
> 只要两边同时出现的值，就以笛卡尔积的方式加入
>
> 如果单边出现则根据连接形式进行处理

笛卡尔积：设左表中键`张三`出现两次，右表中的`张三`也出现两次，那么逐个进行匹配，最后产生的表必然包含`2*2`个姓名为`张三`的行。如图：

![figure 2](https://www.jianshu.com/p/23a2f0804b82)

其中左连接和右连接是等价的，由于它们的结果中的键是被一侧的表确定的，需要区分时是当需要方向性地添加到目标表

### 2. 值连接

区分于根据某一列的值来连接,可以通过几列值的组合进行连接，这种**基于值的连接**在`pandas`中可以由`merge`函数实现，例如第一张图的左连接：

In [4]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
print(df1)
print()
print(df2)
print('--------------')
df1.merge(df2, on='Name', how='left')  # 左连接

        Name  Age
0  San Zhang   20
1      Si Li   30

      Name Gender
0    Si Li      F
1  Wu Wang      M
--------------


,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


当想要连接的列不具备相同的列名，可以通过`left_on`和`right_on`指定：

In [5]:
df1 = pd.DataFrame({'df1_name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'df2_name':['Si Li','Wu Wang'], 'Gender':['F','M']})
print(df1)
print()
print(df2)
print('------------')
df1.merge(df2, left_on='df1_name', right_on='df2_name', how='left')

    df1_name  Age
0  San Zhang   20
1      Si Li   30

  df2_name Gender
0    Si Li      F
1  Wu Wang      M
------------


,df1_name,Age,df2_name,Gender
0,San Zhang,20,NaN,NaN
1,Si Li,30,Si Li,F


如果两个表中的列出现了重复的列名，那么可以通过`suffixes`参数分别指定列名

In [7]:
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70]})
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80]})
print(df1)
print()
print(df2)
print('------------')
df1.merge(df2, on='Name', how='left', suffixes=['_Chinese','_Math'])

        Name  Grade
0  San Zhang     70

        Name  Grade
0  San Zhang     80
------------


,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80


考虑出现重复元素：例如两位同学来自不同的班级，但是姓名相同，这种时候就要指定`on`参数为多个列使得正确连接：

In [11]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['two', 'one']})
print(df1)
print()
print(df2)
print()
tmp = df1.merge(df2, on='Name', how='left') 
print('----- wrong ---------\n {} \n'.format(tmp))
tmp = df1.merge(df2, on=['Name', 'Class'], how='left') # 正确的结果
print('----- right ---------\n {} \n'.format(tmp))


        Name  Age Class
0  San Zhang   20   one
1  San Zhang   21   two

        Name Gender Class
0  San Zhang      F   two
1  San Zhang      M   one

----- wrong ---------
         Name  Age Class_x Gender Class_y
0  San Zhang   20     one      F     two
1  San Zhang   20     one      M     one
2  San Zhang   21     two      F     two
3  San Zhang   21     two      M     one 

----- right ---------
         Name  Age Class Gender
0  San Zhang   20   one      M
1  San Zhang   21   two      F 



进行基于唯一性的连接下，如果键不是唯一的，那么结果就会产生问题。当有几十万到上百万行的进行合并时，如果想要保证唯一性

1. 除了用`duplicated`检查

2. `merge`中也提供的`validate`检查唯一性模式。共有三种模式：

> `1:1`:左右表的键都是唯一
>
> `1:m`：左表键唯一和
>
> `m:1`：右表键唯一。

#### 【练一练】
上面以多列为键的例子中，错误写法显然是一种多对多连接，而正确写法是一对一连接，请修改原表，使得以多列为键的正确写法能够通过`validate='1:m'`的检验，但不能通过`validate='m:1'`的检验。


In [18]:
# 实例5.2：正确应用 - 指定唯一值
df1 = pd.DataFrame({'no':[11,12,13],'name' : ['Tom','Jim','Jim'], 'age' : [13, 13,14]})
df2 = pd.DataFrame({'no' : [11,12,13], 'weight': [30,40,40]})
print(df1)
print()
print(df2)
df1.merge(df2, on='no', how='outer', validate="1:1")#no为唯一值


   no name  age
0  11  Tom   13
1  12  Jim   13
2  13  Jim   14

   no  weight
0  11      30
1  12      40
2  13      40


,no,name,age,weight
0,11,Tom,13,30
1,12,Jim,13,40
2,13,Jim,14,40


In [22]:
#　要求：　多列为键的正确写法
#        能通过validate='1:m'的检验 --> df1 的列 unique
#        不能通过validate='m:1'的检验 --> df2 的列 disunique
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['one', 'one']})  # disunique
print(df1)
print()
print(df2)
tmp = df1.merge(df2, on=['Name', 'Class'], how='left', validate='1:m')
print('--------- 1:m ------------\n{}\n'.format(tmp))  # 能通过说明 df1 -- unique

try:
    tmp = df1.merge(df2, on=['Name', 'Class'], how='left', validate='m:1')
except Exception as e:
    print('不能通过说明 df2 不是 unique ，我以为是呢')
    print(e)

        Name  Age Class
0  San Zhang   20   one
1  San Zhang   21   two

        Name Gender Class
0  San Zhang      F   one
1  San Zhang      M   one
--------- 1:m ------------
        Name  Age Class Gender
0  San Zhang   20   one      F
1  San Zhang   20   one      M
2  San Zhang   21   two    NaN

不能通过说明 df2 不是 unique ，我以为是呢
Merge keys are not unique in right dataset; not a many-to-one merge


### 3. 索引连接

把索引当作键 **与** 值连接本质上没有区别

>`pandas` -- `join`: 索引连接:参数更少
>
>`on`： 索引名，单层索引时省略参数 --> 按照当前索引连接
>
>`how`：
>
>`lsuffix` -- `rsuffix`: 对重复的列指定左右后缀

In [23]:
df1 = pd.DataFrame({'Age':[20,30]}, index=pd.Series(['San Zhang','Si Li'],name='Name'))
df2 = pd.DataFrame({'Gender':['F','M']}, index=pd.Series(['Si Li','Wu Wang'],name='Name'))
print(df1)
print()
print(df2)
print('------------')
df1.join(df2, how='left')

           Age
Name          
San Zhang   20
Si Li       30

        Gender
Name          
Si Li        F
Wu Wang      M
------------


,Age,Gender
Name,,
San Zhang,20,NaN
Si Li,30,F


仿照第2小节的例子，写出语文和数学分数合并的`join`版本：

In [24]:
df1 = pd.DataFrame({'Grade':[70]}, 
                   index=pd.Series(['San Zhang'], 
                   name='Name'))
df2 = pd.DataFrame({'Grade':[80]}, 
                   index=pd.Series(['San Zhang'], 
                   name='Name'))
print(df1)
print()
print(df2)
print('------------')
df1.join(df2, how='left', lsuffix='_Chinese', rsuffix='_Math')

           Grade
Name            
San Zhang     70

           Grade
Name            
San Zhang     80
------------


,Grade_Chinese,Grade_Math
Name,,
San Zhang,70,80


对比`merge`中以多列为键的操作，`join`需要使用多级索引

例如在`merge`中的最后一个例子可以如下写出：

In [26]:
df1 = pd.DataFrame({'Age':[20,21]}, 
                   index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],
                                                    ['one', 'two']], 
                   names=('Name','Class')))
df2 = pd.DataFrame({'Gender':['F', 'M']}, 
                   index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],
                                                    ['two', 'one']], 
                   names=('Name','Class')))
print(df1)
print()
print(df2)
print('------------')
df1.join(df2)

                 Age
Name      Class     
San Zhang one     20
          two     21

                Gender
Name      Class       
San Zhang two        F
          one        M
------------


Age Gender
Name      Class            
San Zhang one     20      M
          two     21      F

## 二、方向连接
### 1. concat
关系型连接，其中最重要的参数是`on`和`how`

以哪一列为键来合并有时候没那么重要 -- 》 只是希望**把两个表或者多个表按照纵向或者横向拼接** --- `pandas`中提供了`concat`函数来实现。

在`concat`中，最常用参数:
> `axis`: 拼接方向， 默认为0，纵向样本拼接；1--横向拼接字段、特征
>
> `join`: 连接形式
> 
> `keys`: 新表中指示来自于哪一张旧表的名字
>
> Attention == 此 join 与彼 join 无关


In [27]:
# 纵向
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})

print(df1)
print()
print(df2)
print('------------')
pd.concat([df1, df2])

        Name  Age
0  San Zhang   20
1      Si Li   30

      Name  Age
0  Wu Wang   40
------------


,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


In [28]:
# 横向
df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})
print(df1)
print()
print(df2)
print()
print(df3)

print('------------')
pd.concat([df1, df2, df3], 1)

        Name  Age
0  San Zhang   20
1      Si Li   30

   Grade
0     80
1     90

  Gender
0      M
1      F
------------


,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


:`concat`是处理关系型合并，但仍关于索引进行连接

纵向拼接：列索引对齐:

> 默认`join=outer` --> (全连接)表示保留所有的列，并将不存在的值设为缺失；
> 
> `join=inner` --> 表示保留两个表都出现过的列

横向拼接：行索引对齐
>`join`参数同上

In [29]:
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Gender':['M']})
print(df1)
print()
print(df2)
pd.concat([df1, df2])

        Name  Age
0  San Zhang   20
1      Si Li   30

      Name Gender
0  Wu Wang      M


,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,NaN
0,Wu Wang,NaN,M


In [30]:
df2 = pd.DataFrame({'Grade':[80, 90]}, index=[1, 2])
print(df1)
print()
print(df2)
pd.concat([df1, df2], 1)

        Name  Age
0  San Zhang   20
1      Si Li   30

   Grade
1     80
2     90


,Name,Age,Grade
0,San Zhang,20.0,NaN
1,Si Li,30.0,80.0
2,NaN,NaN,90.0


In [31]:
print(df1)
print()
print(df2)
pd.concat([df1, df2], axis=1, join='inner')

        Name  Age
0  San Zhang   20
1      Si Li   30

   Grade
1     80
2     90


,Name,Age,Grade
1,Si Li,30,80


对使用多表直接的方向合并（尤其是横向的合并）：

先用 `reset_index` 方法恢复默认整数索引再进行合并

防止**索引的误对齐** 和 **重复索引的笛卡尔积**

`keys`参数的使用场景: 多个表合并,且用户仍然想要知道新表中的数据来自于哪个原表，这时使用`keys`参数产生多级索引进行标记


In [35]:
# 例如，第一个表中都是一班的同学，而第二个表中都是二班的同学：df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'],'Age':[21]})
print(df1)
print()
print(df2)
pd.concat([df1, df2], keys=['one', 'two'])

        Name  Age
0  San Zhang   20
1      Si Li   21

      Name  Age
0  Wu Wang   21


Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

### 2. 序列与表的合并
> `append`:序列追加到表的行末：
>
> ---- 对默认整数序列的索引 -- `ignore_index=True`对新序列对应索引自动标号，否则必须对`Series`指定`name`属性。
> 
>
> `assign`:序列追加到表的列末

In [36]:
s = pd.Series(['Wu Wang', 21], index = df1.columns)
print(s)
df1.append(s, ignore_index=True)

Name    Wu Wang
Age          21
dtype: object


,Name,Age
0,San Zhang,20
1,Si Li,21
2,Wu Wang,21


`assign`添加列 == `df['new_col'] = ...`

但是`[]`会直接在原表上进行改动，而`assign`返回临时副本：

In [37]:
s = pd.Series([80, 90])
print(df1.assign(Grade=s))
s

        Name  Age  Grade
0  San Zhang   20     80
1      Si Li   21     90


0    80
1    90
dtype: int64

In [22]:
df1['Grade'] = s
df1

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


## 三、类连接操作
`pandas`: 对两个表进行"某些操作"，这里把它们统称为类连接操作。

### 1. 比较

`compare`是在`1.1.0`后引入：比较两个表或者序列的不同处并将其汇总展示：

In [39]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'Name':['San Zhang', 'Li Si', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'Three']})
print(df1)
print()
print(df2)
print('------------')
df1.compare(df2)  # 完全一样的只有 0

        Name  Age  Class
0  San Zhang   20    one
1      Si Li   21    two
2    Wu Wang   21  three

        Name  Age  Class
0  San Zhang   20    one
1      Li Si   21    two
2    Wu Wang   21  Three
------------


Name         Class       
    self  other   self  other
1  Si Li  Li Si    NaN    NaN
2    NaN    NaN  three  Three

对返回的不同行，若相同则会被填充为缺失值`NaN`
> `other`： 传入的参数表
>
> `self`： 被调用的表自身。

若需要完整显示表中所有元素比较情况，可以设置`keep_shape=True`

In [40]:
df1.compare(df2, keep_shape=True)

Name         Age        Class       
    self  other self other   self  other
0    NaN    NaN  NaN   NaN    NaN    NaN
1  Si Li  Li Si  NaN   NaN    NaN    NaN
2    NaN    NaN  NaN   NaN  three  Three

### 2. 组合

`combine`函数能够让两张表按照一定的规则进行组合，在进行规则比较时会自动进行列索引的对齐。对于传入的函数而言，每一次操作中输入的参数是来自两个表的同名`Series`，依次传入的列是两个表列名的并集，例如下面这个例子会依次传入`A,B,C,D`四组序列，每组为左右表的两个序列。同时，进行`A`列比较的时候，`s1`指代的就是一个全空的序列，因为它在被调用的表中并不存在，并且来自第一个表的序列索引会被`reindex`成两个索引的并集。具体的过程可以通过在传入的函数中插入适当的`print`方法查看。

下面的例子表示选出对应索引位置较小的元素：

In [25]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)
    res = s1.where(s1<s2, s2)
    res = res.mask(s1.isna()) # isna表示是否为缺失值，返回布尔序列
    return res
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


#### 【练一练】
请在上述代码的基础上修改，保留`df2`中4个未被`df1`替换的相应位置原始值。
#### 【END】
此外，设置`overtwrite`参数为`False`可以保留$\color{red}{被调用表}$中未出现在传入的参数表中的列，而不会设置未缺失值：

In [26]:
df1.combine(df2, choose_min, overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


#### 【练一练】
除了`combine`之外，`pandas`中还有一个`combine_first`方法，其功能是在对两张表组合时，若第二张表中的值在第一张表中对应索引位置的值不是缺失状态，那么就使用第一张表的值填充。下面给出一个例子，请用`combine`函数完成相同的功能。
#### 【END】

In [27]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


## 四、练习
### Ex1：美国疫情数据集

现有美国4月12日至11月16日的疫情报表，请将`New York`的`Confirmed, Deaths, Recovered, Active`合并为一张表，索引为按如下方法生成的日期字符串序列：

In [32]:
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2) +'-'+ date.dt.day.astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()
date[:5]

['04-12-2020', '04-13-2020', '04-14-2020', '04-15-2020', '04-16-2020']

### Ex2：实现join函数

请实现带有`how`参数的`join`函数

* 假设连接的两表无公共列
* 调用方式为 `join(df1, df2, how="left")`
* 给出测试样例